## Cow parameter model with retargeting
We use SMAL (Skinned Multi-Animal Linear model) as the raw parameter models.
![](../data/smal.png)
### Task
Input: SMAL parameters
Output: High resolution cow model

pipeline: SMAL parameter -> registed smal raw model -> chamfer distance using point cloud register -> caculate the intepolation matrix -> retargeting -> skinning

### Model
The SMAL model is a function $M(\beta, \theta, \gamma)$ of shape $\beta$, pose $\theta$ and translation $\gamma$. 
 - $\beta$ is the PCA shape space
 - $\theta\in \mathbb R^{3N}$, where $N=33$ is the relative rotation of 33 joints in the kinematic tree
 - $\gamma$ is the global translation applied to the root joint 
 - Input: $\beta,\theta,\gamma$
 - Output: $M(\beta,\theta,\gamma)$ is a 3D model.
- To be specific: $M(\beta,\theta,\gamma)=\text{LBS}\left(\Theta(\theta,\gamma),\overline{T}(\beta)\right)$
  - LBS is linear blend shape model, taking the template and pose
  - $\overline{T}(\beta)$ is the template of registered pose taking the shape vector $\beta$
  - $\Theta$ is the per-joint transform taking the relative pose vector $\theta$ from kinematic tree and a global tranformation $\gamma$ to the root joint.
  
## Demo

### Retargeting
#### Point cloud registering
Problem Definition:

Given two point clouds:
 - Source point cloud $S = \{s_1, ..., s_n\}$
 - Target point cloud $T = \{t_1, ..., t_m\}$.
  
The goal is to find the optimal rigid transformation (rotation $R$ and translation $t$) that aligns the source point cloud with the target point cloud.

**Transformation Model**

The rigid transformation is defined as:
$s'_i = Rs_i + t$ where $R$ is a 3×3 rotation matrix,
$t$ is a 3×1 translation vector,
$s_i$ is a point in the source cloud,
$s'_i$ is the transformed point.

We uses the SO(3) exponential map to parameterize rotation:
$R = \exp(\omega^\wedge)$ where:
$\omega$ is a 3D rotation vector,
$\exp$ is the matrix exponential,
$^\wedge$ converts a 3D vector to a 3×3 skew-symmetric matrix.

**Chamfer Distance as loss function**

The algorithm uses Chamfer Distance as the loss function:
$$L(S, T) = \frac{1}{|S|}\sum_{s \in S}\min_{t \in T}||s - t||^2 + \frac{1}{|T|}\sum_{t \in T}\min_{s \in S}||s - t||^2$$

We aimed to use point registering algorithm to achieve the retargeting.
 - Motivation: input ($\beta$,$\Theta$), output: fine mesh
 - The backward gradient to reduce the loss:
$$\frac{\partial{\text{chamfer\_distance}}}{\partial \{\beta,\Theta\}}=\frac{\partial \text{chamfer\_distance}}{\partial \text{fine\_mesh}}\cdot\frac{\partial \text{fine\_mesh}}{\partial \text{SMAL\_model}}\cdot\frac{\partial \text{SMAL\_model}}{\partial \{\beta,\Theta\}}$$
 - The fitting step will output the $\{\beta,\Theta\}$, we can then use the gradient from SMAL and the interpolation matrix between SMAL raw mesh and the fine mesh($\frac{\partial \text{fine\_mesh}}{\partial \text{SMAL\_model}}$) from the same pose as well as the gradient from point cloud registering ($\frac{\partial \text{chamfer\_distance}}{\partial \text{fine\_mesh}}$) to obtain the retargeted fine mesh.

**Post process on skinning mesh**
We use a Laplacian energy as regular term to optimize the distortion after skinning.

  